# The Battle of the Neighborhoods - Week 2

### Import necessary libraries

In [2]:
import numpy as np 
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
import json 
!conda install -c conda-forge geopy --yes 
from geopy.geocoders import Nominatim 
import requests 
from pandas.io.json import json_normalize 
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans
!conda install -c conda-forge folium=0.5.0 --yes 
import folium

print('Done! Libraries imported.')

Solving environment: done

## Package Plan ##

  environment location: /opt/conda/envs/Python36

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ca-certificates-2020.6.20  |       hecda079_0         145 KB  conda-forge
    openssl-1.1.1g             |       h516909a_1         2.1 MB  conda-forge
    geographiclib-1.50         |             py_0          34 KB  conda-forge
    geopy-2.0.0                |     pyh9f0ad1d_0          63 KB  conda-forge
    certifi-2020.6.20          |   py36h9f0ad1d_0         151 KB  conda-forge
    python_abi-3.6             |          1_cp36m           4 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         2.5 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.50-py_0          conda-forge
    geopy:           

## New York, New York

### Download datasets for NYC analysis and transform it into a pandas dataframe

In [3]:
#First we get the data table for the postal codes of NYC neighborhoods.
PostCodes_NY= pd.read_html('https://en.wikipedia.org/wiki/Neighborhoods_in_New_York_City', header=0)
df_NY= PostCodes_NY[0]
df_NY

,Community Board(CB),Areakm2,Pop.Census2010,Pop./km2,Neighborhoods
0,Bronx CB 1,7.17,91497,12761,"Melrose, Mott Haven, Port Morris"
1,Bronx CB 2,5.54,52246,9792,"Hunts Point, Longwood"
2,Bronx CB 3,4.07,79762,19598,"Claremont, Concourse Village, Crotona Park, Mo..."
3,Bronx CB 4,5.28,146441,27735,"Concourse, Highbridge"
4,Bronx CB 5,3.55,128200,36145,"Fordham, Morris Heights, Mount Hope, Universit..."
5,Bronx CB 6,4.01,83268,20765,"Bathgate, Belmont, East Tremont, West Farms"
6,Bronx CB 7,4.84,139286,28778,"Bedford Park, Norwood, University Heights"
7,Bronx CB 8,8.83,101731,11521,"Fieldston, Kingsbridge, Kingsbridge Heights, M..."
8,Bronx CB 9,12.41,172298,13884,"Bronx River, Bruckner, Castle Hill, Clason Poi..."
9,Bronx CB 10,16.76,120392,7183,"City Island, Co-op City, Locust Point, Pelham ..."


In [4]:
#Rename the columns
df_NY= df_NY.rename(columns= {'Community Board(CB)':'Borough', 'Areakm2':'Area (in km^2)', 'Pop.Census2010':'Population (2010)', 'Pop./km2':'Population (in km^2)'})
df_NY= df_NY.drop([59,1]) #I drop the general New York City row at the bottom of the dataframe
df_NY.head()

,Borough,Area (in km^2),Population (2010),Population (in km^2),Neighborhoods
0,Bronx CB 1,7.17,91497,12761,"Melrose, Mott Haven, Port Morris"
2,Bronx CB 3,4.07,79762,19598,"Claremont, Concourse Village, Crotona Park, Mo..."
3,Bronx CB 4,5.28,146441,27735,"Concourse, Highbridge"
4,Bronx CB 5,3.55,128200,36145,"Fordham, Morris Heights, Mount Hope, Universit..."
5,Bronx CB 6,4.01,83268,20765,"Bathgate, Belmont, East Tremont, West Farms"


In [5]:
Postal_Codes= ['10453', '10453', '10453', '10453', '10453', '10453', '10453', '10453', '10453', '10453', '10453', 
              '11212', '11212', '11212', '11212', '11212', '11212', '11212', '11212', '11212', '11212', '11212', 
              '11212', '11212', '11212', '11212', '11212', '11212', '11212', 
              '10026', '10026', '10026', '10026', '10026', '10026', '10026', '10026', '10026', '10026', '10026', 
              '10026', 
              '11361', '11361', '11361', '11361', '11361', '11361', '11361', '11361', '11361', '11361', '11361', 
              '11361', '11361', '11361', 
              '10302', '10302', '10302', 
              ]
df_NY['Postal Codes']= Postal_Codes
df_NY

,Borough,Area (in km^2),Population (2010),Population (in km^2),Neighborhoods,Postal Codes
0,Bronx CB 1,7.17,91497,12761,"Melrose, Mott Haven, Port Morris",10453
2,Bronx CB 3,4.07,79762,19598,"Claremont, Concourse Village, Crotona Park, Mo...",10453
3,Bronx CB 4,5.28,146441,27735,"Concourse, Highbridge",10453
4,Bronx CB 5,3.55,128200,36145,"Fordham, Morris Heights, Mount Hope, Universit...",10453
5,Bronx CB 6,4.01,83268,20765,"Bathgate, Belmont, East Tremont, West Farms",10453
6,Bronx CB 7,4.84,139286,28778,"Bedford Park, Norwood, University Heights",10453
7,Bronx CB 8,8.83,101731,11521,"Fieldston, Kingsbridge, Kingsbridge Heights, M...",10453
8,Bronx CB 9,12.41,172298,13884,"Bronx River, Bruckner, Castle Hill, Clason Poi...",10453
9,Bronx CB 10,16.76,120392,7183,"City Island, Co-op City, Locust Point, Pelham ...",10453
10,Bronx CB 11,9.32,113232,12149,"Allerton, Bronxdale, Indian Village, Laconia, ...",10453


As you can see on the table above, there are 5 boroughs in New York: Bronx, Brooklyn, Manhattan, Queens, and Staten Island. 

__*NOTE*__: These postal codes were obtained from https://www.health.ny.gov/statistics/cancer/registry/appendix/neighborhoods.htm ; Only one postal code for each borough (from the central part of the borough) was recorded for simplicity and visualization purposes.

In [6]:
df_NY.shape

(58, 6)

### Get the sum of the population

*_Note:_* This population value is from 2010, but later on in this code you can see and compare with the current population (from census.gov).

In [7]:
total_pop= df_NY['Population (2010)'].sum()
total_pop

8006205

### Census data for average income, businesses, and retail sales

Obtain and read in data from the NY census to analyze in the report how population, businesses, and the economy of the city might influence the future of the clothing store. 

In [8]:
census_NY= pd.read_html('https://www.census.gov/quickfacts/fact/table/newyorkcitynewyork/PST040219#qf-flag-X')
census_NY

[  All Topics Population Age and Sex Race and Hispanic Origin Population Characteristics Housing Families & Living Arrangements Computer and Internet Use Education Health Economy Transportation Income & Poverty Businesses Geography  \
 0  Population estimates base, April 1, 2010, (V2019)                                                                                                                                                                                     
 
    New York city, New York  
 0                  8175031  ,
                                            Population Unnamed: 1
 0         Population estimates, July 1, 2019, (V2019)    8336817
 1   Population estimates base, April 1, 2010, (V2019)    8175031
 2   Population, percent change - April 1, 2010 (es...       2.0%
 3                   Population, Census, April 1, 2010    8175133
 4                                         Age and Sex        NaN
 5                      Persons under 5 years, percent     6.5%
 6   

As you can see above, the estimate for population from census.gov in 2010 is around 8175031 while our sum of the population by borough is a just a little less (around 8006205). The population from census.gov in 2019 is higher, at around 8336817. We can calculate the population growth rate to determine how many future clients there might be.

In [9]:
pop2010= 8175031
pop2019= 8336817
pop_growth= ((pop2019-pop2010)/(pop2010))*100
print('The population growth rate (from 2010-2019) was around:', pop_growth, '%')

The population growth rate (from 2010-2019) was around: 1.9790261345797957 %


In 9 years the population grew by almost 2%. This means we could approximate the population in 9 years to be around: 

In [10]:
futurepop= (pop2019*(pop_growth/100))+pop2019
futurepop

8501804.787222091

### Data for costs of living in NY

This information can tell us how much people need in order to eat, pay rent, commute, and spend on other necessities. If the income we see is much higher than what the average person needs, then they're more likely to spend more money on luxury items such as this clothing store.

In [11]:
livingNY= pd.read_html('https://www.numbeo.com/cost-of-living/in/New-York')
livingNY

[    0                                                  1
 0 NaN  whenDocReady(function() {  jQuery('#city_selec...,     0      1
 0 NaN  Tweet,                                           Restaurants     [ Edit ]  \
 0                        Meal, Inexpensive Restaurant      21.00 $   
 1   Meal for 2 People, Mid-range Restaurant, Three...     100.00 $   
 2      McMeal at McDonalds (or Equivalent Combo Meal)       9.25 $   
 3                      Domestic Beer (1 pint draught)       7.00 $   
 4                  Imported Beer (12 oz small bottle)       8.00 $   
 5                                Cappuccino (regular)       4.90 $   
 6                     Coke/Pepsi (12 oz small bottle)       2.19 $   
 7                          Water (12 oz small bottle)       1.77 $   
 8                                             Markets     [ Edit ]   
 9                          Milk (regular), (1 gallon)       4.42 $   
 10                   Loaf of Fresh White Bread (1 lb)       3.36 $   
 11 

### Create a map of NYC Boroughs

In [12]:
from IPython.display import display

In [13]:
latitudeNYC= 40.730610
longitudeNYC= -73.935242

latBronx= 40.8448
lonBronx= -73.8648
latBrook= 40.6782
lonBrook= -73.9442
latManh= 40.7831
lonManh= -73.9712
latQueens= 40.7282
lonQueens= -73.7949
latSI= 40.5795
lonSI= -74.1502

map_NY = folium.Map(location=[latitudeNYC, longitudeNYC], zoom_start=12)

folium.Marker([latBronx, lonBronx],
              popup='Bronx',
              icon=folium.Icon(color='green')
             ).add_to(map_NY)
folium.Marker([latBrook, lonBrook],
              popup='Brooklyn',
              icon=folium.Icon(color='red')
             ).add_to(map_NY)
folium.Marker([latManh, lonManh],
              popup='Manhattan',
              icon=folium.Icon(color='purple')
             ).add_to(map_NY)
folium.Marker([latQueens, lonQueens],
              popup='Queens',
              icon=folium.Icon(color='blue')
             ).add_to(map_NY)
folium.Marker([latSI, lonSI],
              popup='Staten Island',
              icon=folium.Icon(color='pink')
             ).add_to(map_NY)

map_NY

In [288]:
CLIENT_ID = '1IVZFEEO0D5XWSHQUVP0UIMMFSPSPDXWYFMCCFMFEMEAMLXJ' 
CLIENT_SECRET = 'LRRZSM13LBPVZ1PHJQX0PDZML2C1QJQM2HNQ2JEXSWAK4S2V' 
VERSION = '20180604'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)


Your credentails:
CLIENT_ID: 1IVZFEEO0D5XWSHQUVP0UIMMFSPSPDXWYFMCCFMFEMEAMLXJ
CLIENT_SECRET:LRRZSM13LBPVZ1PHJQX0PDZML2C1QJQM2HNQ2JEXSWAK4S2V


### Search on Foursquare for every popular clothing venue in each NY borough

In [369]:
address = 'New York, NY'
latitude= 40.7493
longitude= -73.9893
print(latitude, longitude)

40.7493 -73.9893


In [370]:
search_query = 'mall'
radius = 500
print(' Searching for:',search_query +  '.... OK!')

 Searching for: mall.... OK!


In [371]:
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius, LIMIT)
url

'https://api.foursquare.com/v2/venues/search?client_id=1IVZFEEO0D5XWSHQUVP0UIMMFSPSPDXWYFMCCFMFEMEAMLXJ&client_secret=LRRZSM13LBPVZ1PHJQX0PDZML2C1QJQM2HNQ2JEXSWAK4S2V&ll=40.7493,-73.9893&v=20180604&query=mall&radius=500&limit=30'

In [372]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5f371ebaccdcdc5e3f22532e'},
 'response': {'venues': [{'id': '4a19bdd6f964a520477a1fe3',
    'name': 'Manhattan Mall',
    'location': {'address': '100 W 33rd St',
     'crossStreet': 'b/t Avenue Of The Americas & West 33rd St',
     'lat': 40.749055139214676,
     'lng': -73.98895146714219,
     'labeledLatLngs': [{'label': 'entrance',
       'lat': 40.749391,
       'lng': -73.989106}],
     'distance': 40,
     'postalCode': '10001',
     'cc': 'US',
     'neighborhood': 'New York, NY',
     'city': 'New York',
     'state': 'NY',
     'country': 'United States',
     'formattedAddress': ['100 W 33rd St (b/t Avenue Of The Americas & West 33rd St)',
      'New York, NY 10001',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png

In [373]:
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4a19bdd6f964a520477a1fe3,100 W 33rd St,US,New York,United States,b/t Avenue Of The Americas & West 33rd St,40,[100 W 33rd St (b/t Avenue Of The Americas & W...,"[{'label': 'entrance', 'lat': 40.749391, 'lng'...",40.749055,-73.988951,"New York, NY",10001,NY,Manhattan Mall,v-1597448348,NaN
1,"[{'id': '4bf58dd8d48988d178941735', 'name': 'D...",False,4bbf6366006dc9b6627ffc3f,901 Ave of the Americas,US,New York,United States,W 33rd St,118,"[901 Ave of the Americas (W 33rd St), New York...","[{'label': 'display', 'lat': 40.7485880697979,...",40.748588,-73.988263,NaN,10001-3511,NY,Manhattan Mall Dental,v-1597448348,NaN
2,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",False,4a802155f964a520a9f41fe3,875 Avenue of the Americas,US,New York,United States,at W 31st St,3,"[875 Avenue of the Americas (at W 31st St), Ne...","[{'label': 'display', 'lat': 40.74927198, 'lng...",40.749272,-73.989278,NaN,10001,NY,Starbucks,v-1597448348,NaN
3,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",False,4bcc8c5d937ca5931349ab92,901 6th Ave.,US,New York,United States,at W 32nd St.,4,"[901 6th Ave. (at W 32nd St.), New York, NY 10...","[{'label': 'display', 'lat': 40.74931371717206...",40.749314,-73.989344,NaN,10001,NY,Strawberry,v-1597448348,NaN
4,"[{'id': '4bf58dd8d48988d111951735', 'name': 'J...",False,4d921bb6f5388cfac7e8c43d,NaN,US,NaN,United States,NaN,234,"[New York, United States]","[{'label': 'display', 'lat': 40.75036096969143...",40.750361,-73.991706,NaN,NaN,New York,Golden Mall Jewelry,v-1597448348,NaN
5,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",False,4bd5c40b5631c9b61ea4a430,901 6th Ave,US,New York,United States,btwn W 34th & W 33rd St,18,"[901 6th Ave (btwn W 34th & W 33rd St), New Yo...","[{'label': 'display', 'lat': 40.74918312789088...",40.749183,-73.989451,NaN,10001,NY,Express,v-1597448348,NaN
6,"[{'id': '4bf58dd8d48988d103951735', 'name': 'C...",False,4df10dd881303a516711eec6,901 6th Avenue,US,New York,United States,NaN,26,"[901 6th Avenue, New York, NY 10001, United St...","[{'label': 'display', 'lat': 40.74911014966418...",40.749110,-73.989111,NaN,10001,NY,Virtu,v-1597448348,90491097
7,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",False,4da35b13b3e7236afbfc3079,Broadway btwn 39th & 42nd Streets,US,New York,United States,NaN,520,"[Broadway btwn 39th & 42nd Streets, New York, ...","[{'label': 'display', 'lat': 40.75371457242185...",40.753715,-73.987255,NaN,10018,NY,Broadway Pedestrian Mall - 39th St to 42nd St,v-1597448348,NaN
8,"[{'id': '4bf58dd8d48988d10b951735', 'name': 'V...",False,4a6f3c21f964a520b0d51fe3,1282 Broadway Ste 249,US,New York,United States,at W 33rd St,95,"[1282 Broadway Ste 249 (at W 33rd St), New Yor...","[{'label': 'display', 'lat': 40.74910601610609...",40.749106,-73.988202,NaN,10001,NY,GameStop,v-1597448348,NaN
9,"[{'id': '4bf58dd8d48988d105951735', 'name': 'K...",False,4c694548c946e21e06ade98e,100 W 33rd St,US,New York,United States,Manhattan Mall,52,"[100 W 33rd St (Manhattan Mall), New York, NY ...","[{'label': 'display', 'lat': 40.74903692976218...",40.749037,-73.988779,NaN,10001,NY,P.S. from Aéropostale,v-1597448348,NaN


In [374]:
filtered_columns = ['name', 'categories'] + [col for col in dataframe.columns if col.startswith('location.')] + ['id']
dataframe_filtered = dataframe.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered['categories'] = dataframe_filtered.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered.columns = [column.split('.')[-1] for column in dataframe_filtered.columns]

dataframe_filtered

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,neighborhood,postalCode,state,id
0,Manhattan Mall,Shopping Mall,100 W 33rd St,US,New York,United States,b/t Avenue Of The Americas & West 33rd St,40,[100 W 33rd St (b/t Avenue Of The Americas & W...,"[{'label': 'entrance', 'lat': 40.749391, 'lng'...",40.749055,-73.988951,"New York, NY",10001,NY,4a19bdd6f964a520477a1fe3
1,Manhattan Mall Dental,Dentist's Office,901 Ave of the Americas,US,New York,United States,W 33rd St,118,"[901 Ave of the Americas (W 33rd St), New York...","[{'label': 'display', 'lat': 40.7485880697979,...",40.748588,-73.988263,NaN,10001-3511,NY,4bbf6366006dc9b6627ffc3f
2,Starbucks,Coffee Shop,875 Avenue of the Americas,US,New York,United States,at W 31st St,3,"[875 Avenue of the Americas (at W 31st St), Ne...","[{'label': 'display', 'lat': 40.74927198, 'lng...",40.749272,-73.989278,NaN,10001,NY,4a802155f964a520a9f41fe3
3,Strawberry,Women's Store,901 6th Ave.,US,New York,United States,at W 32nd St.,4,"[901 6th Ave. (at W 32nd St.), New York, NY 10...","[{'label': 'display', 'lat': 40.74931371717206...",40.749314,-73.989344,NaN,10001,NY,4bcc8c5d937ca5931349ab92
4,Golden Mall Jewelry,Jewelry Store,NaN,US,NaN,United States,NaN,234,"[New York, United States]","[{'label': 'display', 'lat': 40.75036096969143...",40.750361,-73.991706,NaN,NaN,New York,4d921bb6f5388cfac7e8c43d
5,Express,Clothing Store,901 6th Ave,US,New York,United States,btwn W 34th & W 33rd St,18,"[901 6th Ave (btwn W 34th & W 33rd St), New Yo...","[{'label': 'display', 'lat': 40.74918312789088...",40.749183,-73.989451,NaN,10001,NY,4bd5c40b5631c9b61ea4a430
6,Virtu,Clothing Store,901 6th Avenue,US,New York,United States,NaN,26,"[901 6th Avenue, New York, NY 10001, United St...","[{'label': 'display', 'lat': 40.74911014966418...",40.749110,-73.989111,NaN,10001,NY,4df10dd881303a516711eec6
7,Broadway Pedestrian Mall - 39th St to 42nd St,Plaza,Broadway btwn 39th & 42nd Streets,US,New York,United States,NaN,520,"[Broadway btwn 39th & 42nd Streets, New York, ...","[{'label': 'display', 'lat': 40.75371457242185...",40.753715,-73.987255,NaN,10018,NY,4da35b13b3e7236afbfc3079
8,GameStop,Video Game Store,1282 Broadway Ste 249,US,New York,United States,at W 33rd St,95,"[1282 Broadway Ste 249 (at W 33rd St), New Yor...","[{'label': 'display', 'lat': 40.74910601610609...",40.749106,-73.988202,NaN,10001,NY,4a6f3c21f964a520b0d51fe3
9,P.S. from Aéropostale,Kids Store,100 W 33rd St,US,New York,United States,Manhattan Mall,52,"[100 W 33rd St (Manhattan Mall), New York, NY ...","[{'label': 'display', 'lat': 40.74903692976218...",40.749037,-73.988779,NaN,10001,NY,4c694548c946e21e06ade98e


In [380]:
dataframe_filtered.name
dataframe_filtered= dataframe_filtered[0:10]

As can be seen from the above table, this isn't the most accurate representation of clothing stores in NYC. You can use this same code to search for specific stores (i.e. Zara, lululemon, Ted Baker, Nike, etc.) or even department stores (i.e. Macy's, Nordstrom, Bloomingdales, Saks Fifth Avenue, etc.). The owners of the store could search for stores that they know have around the same price range as theirs. I specifically used coordinates closer to Manhattan mall to to avoid overcrowding of data.

### Generate map of the New York venues found

In [381]:
venues_map = folium.Map(location=[latitude, longitude], zoom_start=15)

# add a red circle marker to represent New York City
folium.features.CircleMarker(
    [latitude, longitude],
    radius=10,
    color='red',
    popup='New York, New York',
    fill = True,
    fill_color = 'red',
    fill_opacity = 0.6
).add_to(venues_map)

#for lat, lng, label in zip(dataframe_filtered.lat, dataframe_filtered.lng, dataframe_filtered.categories):
#    folium.features.CircleMarker(
#        [lat, lng],
#        radius=5,
#        color='blue',
#        popup=label,
#        fill = True,
#        fill_color='blue',
#        fill_opacity=0.6
    ).add_to(venues_map)

# display map
venues_map

## Miami, Florida

Now that we have analyzed NY, we move on to analyzing Miami, Florida. Then in the report, I will compare both based on the data analyzed here. 

### Download datasets for Miami analysis and transform it into a pandas dataframe

In [143]:
PostCodes_Mia= pd.read_html('https://en.wikipedia.org/wiki/List_of_neighborhoods_in_Miami', header=0)
df_Mia= PostCodes_Mia[0]
df_Mia

,Neighborhood,Demonym,Population2010,Population/Km²,Sub-neighborhoods,Coordinates
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257
5,Coral Way,NaN,35062,4496,"Coral Gate, Golden Pines, Shenandoah, Historic...",25.750-80.283
6,Design District,NaN,3573,3623,NaN,25.813-80.193
7,Downtown,Downtowner,"71,000 (13,635 CBD only)",10613,"Brickell, Central Business District (CBD), Dow...",25.774-80.193
8,Edgewater,NaN,15005,6675,NaN,25.802-80.190
9,Flagami,NaN,50834,5665,"Alameda, Grapeland Heights, and Fairlawn",25.762-80.316


In [141]:
df_Mia.shape

(26, 6)

__*NOTE:*__ Miami does not have "boroughs" as does NYC, so we use its neighborhoods for analysis purposes.

In [149]:
#Rename the columns
df_Mia= df_Mia.rename(columns= {'Population2010':'Population (2010)', 'Population/Km²': 'Population (per Km²)'})
df_Mia.head()

,Neighborhood,Demonym,Population (2010),Population (per Km²),Sub-neighborhoods,Coordinates,Postal Codes
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224,33125
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190,33132
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193,33131
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192,33137
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257,33133


In [150]:
Postal_CodesMia= ['33125', '33132', '33131', '33137', '33133', '33129', '33127', '33128', '33137', 
                  '33125', '33125', '33136', '33127', '33127', '33130', '33136', '33136', '33136', 
                 '33132', '33129', '33138', '33139', '33149', '33125', '33127', '33132']
df_Mia['Postal Codes']= Postal_CodesMia
df_Mia

,Neighborhood,Demonym,Population (2010),Population (per Km²),Sub-neighborhoods,Coordinates,Postal Codes
0,Allapattah,NaN,54289,4401,NaN,25.815-80.224,33125
1,Arts & Entertainment District,NaN,11033,7948,NaN,25.799-80.190,33132
2,Brickell,Brickellite,31759,14541,West Brickell,25.758-80.193,33131
3,Buena Vista,NaN,9058,3540,Buena Vista East Historic District and Design ...,25.813-80.192,33137
4,Coconut Grove,Grovite,20076,3091,"Center Grove, Northeast Coconut Grove, Southwe...",25.712-80.257,33133
5,Coral Way,NaN,35062,4496,"Coral Gate, Golden Pines, Shenandoah, Historic...",25.750-80.283,33129
6,Design District,NaN,3573,3623,NaN,25.813-80.193,33127
7,Downtown,Downtowner,"71,000 (13,635 CBD only)",10613,"Brickell, Central Business District (CBD), Dow...",25.774-80.193,33128
8,Edgewater,NaN,15005,6675,NaN,25.802-80.190,33137
9,Flagami,NaN,50834,5665,"Alameda, Grapeland Heights, and Fairlawn",25.762-80.316,33125


__*NOTE:*__ Postal codes can be found at https://worldpostalcode.com/united-states/florida/miami . Some neighborhoods have more than one postal code but only one was recorded in the table above for simplicity purposes.

In [145]:
df_Mia.shape

(26, 7)

### Census data for average income, businesses, and retail sales

Obtain and read in data from the NY census to analyze in the report how population, businesses, and the economy of the city might influence the future of the clothing store. 

In [146]:
census_Mia= pd.read_html('https://www.census.gov/quickfacts/fact/table/miamicityflorida/POP060210')
census_Mia

[  All Topics Population Age and Sex Race and Hispanic Origin Population Characteristics Housing Families & Living Arrangements Computer and Internet Use Education Health Economy Transportation Income & Poverty Businesses Geography  \
 0                   Population per square mile, 2010                                                                                                                                                                                     
 
    Miami city, Florida  
 0              11135.9  ,
                                            Population Unnamed: 1
 0         Population estimates, July 1, 2019, (V2019)     467963
 1   Population estimates base, April 1, 2010, (V2019)     399481
 2   Population, percent change - April 1, 2010 (es...      17.1%
 3                   Population, Census, April 1, 2010     399457
 4                                         Age and Sex        NaN
 5                      Persons under 5 years, percent     5.8%
 6           

### Estimate population growth for Miami

In [154]:
pop2010_Mia= 399481
pop2019_Mia= 467963
pop_growth_Mia= ((pop2019_Mia-pop2010_Mia)/(pop2010_Mia))*100
print('The population growth rate in Miami (from 2010-2019) was around:', pop_growth_Mia, '%')

The population growth rate in Miami (from 2010-2019) was around: 17.142742708664493 %


In [156]:
futurepop_Mia= (pop2019_Mia*(pop_growth_Mia/100))+pop2019_Mia
print('The future estimated population in Miami in 9 years (assuming the growth rate remains the same) is: ', futurepop_Mia)

The future estimated population in Miami in 9 years (assuming the growth rate remains the same) is:  548184.6930617476


### Data for costs of living in Miami

In [157]:
livingMia= pd.read_html('https://www.numbeo.com/cost-of-living/in/Miami')
livingMia

[    0                                                  1
 0 NaN  whenDocReady(function() {  jQuery('#city_selec...,     0      1
 0 NaN  Tweet,                                           Restaurants     [ Edit ]  \
 0                        Meal, Inexpensive Restaurant      15.00 $   
 1   Meal for 2 People, Mid-range Restaurant, Three...      70.00 $   
 2      McMeal at McDonalds (or Equivalent Combo Meal)       8.00 $   
 3                      Domestic Beer (1 pint draught)       6.00 $   
 4                  Imported Beer (12 oz small bottle)       8.00 $   
 5                                Cappuccino (regular)       4.20 $   
 6                     Coke/Pepsi (12 oz small bottle)       2.07 $   
 7                          Water (12 oz small bottle)       1.73 $   
 8                                             Markets     [ Edit ]   
 9                          Milk (regular), (1 gallon)       3.71 $   
 10                   Loaf of Fresh White Bread (1 lb)       2.75 $   
 11 

### Create a map of Miami

In [368]:
latitudeMia= 25.7617
longitudeMia= -80.1918
map_Mia= folium.Map(location= [latitudeMia, longitudeMia], zoom_start= 12)
map_Mia

### Use Foursquare to find popular clothing venues in Miami

In [271]:
CLIENT_ID = '1IVZFEEO0D5XWSHQUVP0UIMMFSPSPDXWYFMCCFMFEMEAMLXJ' 
CLIENT_SECRET = 'LRRZSM13LBPVZ1PHJQX0PDZML2C1QJQM2HNQ2JEXSWAK4S2V' 
VERSION = '20180605'
LIMIT = 30

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: 1IVZFEEO0D5XWSHQUVP0UIMMFSPSPDXWYFMCCFMFEMEAMLXJ
CLIENT_SECRET:LRRZSM13LBPVZ1PHJQX0PDZML2C1QJQM2HNQ2JEXSWAK4S2V


In [341]:
address2 = 'Miami, FL'
latitude2= 25.6905
longitude2= -80.3125
print(latitude2, longitude2)

25.6905 -80.3125


In [342]:
search_query2 = 'mall'
radius = 1000
print(' Searching for:',search_query +  '.... OK!')

 Searching for: clothing.... OK!


In [343]:
url2 = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, latitude2, longitude2, VERSION, search_query2, radius, LIMIT)
url2

'https://api.foursquare.com/v2/venues/search?client_id=1IVZFEEO0D5XWSHQUVP0UIMMFSPSPDXWYFMCCFMFEMEAMLXJ&client_secret=LRRZSM13LBPVZ1PHJQX0PDZML2C1QJQM2HNQ2JEXSWAK4S2V&ll=25.6905,-80.3125&v=20180604&query=mall&radius=1000&limit=30'

In [344]:
results2 = requests.get(url2).json()
results2

{'meta': {'code': 200, 'requestId': '5f371cfa3694f37b7b31dbe0'},
 'response': {'venues': [{'id': '4b14ca5cf964a5208da623e3',
    'name': 'Dadeland Mall',
    'location': {'address': '7535 N Kendall Dr',
     'crossStreet': 'Dadeland Blvd',
     'lat': 25.69054534138613,
     'lng': -80.3124945912628,
     'labeledLatLngs': [{'label': 'display',
       'lat': 25.69054534138613,
       'lng': -80.3124945912628}],
     'distance': 5,
     'postalCode': '33156',
     'cc': 'US',
     'city': 'Miami',
     'state': 'FL',
     'country': 'United States',
     'formattedAddress': ['7535 N Kendall Dr (Dadeland Blvd)',
      'Miami, FL 33156',
      'United States']},
    'categories': [{'id': '4bf58dd8d48988d1fd941735',
      'name': 'Shopping Mall',
      'pluralName': 'Shopping Malls',
      'shortName': 'Mall',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/shops/mall_',
       'suffix': '.png'},
      'primary': True}],
    'referralId': 'v-1597447855',
    'hasPerk': Fal

In [345]:
venues2 = results2['response']['venues']

# tranform venues into a dataframe
dataframe2 = json_normalize(venues2)
dataframe2

,categories,hasPerk,id,location.address,location.cc,location.city,location.country,location.crossStreet,location.distance,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4b14ca5cf964a5208da623e3,7535 N Kendall Dr,US,Miami,United States,Dadeland Blvd,5,"[7535 N Kendall Dr (Dadeland Blvd), Miami, FL ...","[{'label': 'display', 'lat': 25.69054534138613...",25.690545,-80.312495,NaN,33156,FL,Dadeland Mall,v-1597447855,NaN
1,"[{'id': '4bf58dd8d48988d1ce941735', 'name': 'S...",False,52694f11498e980ab15d1af8,7535 N Kendall Dr,US,Kendall,United States,NaN,27,"[7535 N Kendall Dr, Kendall, FL 33156, United ...","[{'label': 'display', 'lat': 25.69064, 'lng': ...",25.690640,-80.312280,Dadeland Mall,33156,FL,SHRIMP HOUSE - Seafood Pasta - Dadeland Mall,v-1597447855,91044305
2,"[{'id': '4bf58dd8d48988d107951735', 'name': 'S...",False,568435fe498e546e328d3958,7535 N Kendall Dr.,US,Miami,United States,NaN,102,"[7535 N Kendall Dr., Miami, FL 33156, United S...","[{'label': 'display', 'lat': 25.6895901, 'lng'...",25.689590,-80.312345,NaN,33156,FL,Donald J Pliner Miami - Dadeland Mall,v-1597447855,NaN
3,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4bc47d42920eb71304c51e2c,7700 N Kendall Dr,US,Miami,United States,NaN,793,"[7700 N Kendall Dr, Miami, FL 33156, United St...","[{'label': 'display', 'lat': 25.68800747614737...",25.688007,-80.319910,NaN,33156,FL,Greenery Mall,v-1597447855,NaN
4,"[{'id': '4bf58dd8d48988d117951735', 'name': 'C...",False,5e616b8b35ec690008a119a6,7535 N Kendall Dr # K148,US,Miami,United States,NaN,6,"[7535 N Kendall Dr # K148, Miami, FL 33156, Un...","[{'label': 'display', 'lat': 25.690559, 'lng':...",25.690559,-80.312484,NaN,33156,FL,Sugar King Dadeland Mall,v-1597447855,NaN
5,"[{'id': '4bf58dd8d48988d1f2941735', 'name': 'S...",False,5332f34e498e79e651781035,NaN,US,Kendall,United States,NaN,48,"[Kendall, FL, United States]","[{'label': 'display', 'lat': 25.69010645150274...",25.690106,-80.312696,NaN,NaN,FL,fit2run dadeland mall,v-1597447855,NaN
6,"[{'id': '4bf58dd8d48988d108951735', 'name': 'W...",False,59e2afc20d173f42885d36f3,72 Avenue,US,Miami,United States,Kendall drive,94,"[72 Avenue (Kendall drive), Miami, FL 33156, U...","[{'label': 'display', 'lat': 25.691221, 'lng':...",25.691221,-80.312008,NaN,33156,FL,New York & Company - Dadeland Mall,v-1597447855,NaN
7,"[{'id': '4bf58dd8d48988d122951735', 'name': 'E...",False,4e15019c52b1b9e5643ce942,7535 N Kendall Dr.,US,Miami,United States,NaN,182,"[7535 N Kendall Dr., Miami, FL 33156, United S...","[{'label': 'display', 'lat': 25.69046031590772...",25.690460,-80.314322,NaN,33156,FL,Cellairis - Dadeland Mall,v-1597447855,347456342
8,"[{'id': '4bf58dd8d48988d1fd941735', 'name': 'S...",False,4f1a2db07beb3367ac3b4580,7535 N Kendall Dr,US,Miami,United States,NaN,121,"[7535 N Kendall Dr, Miami, FL 33156, United St...","[{'label': 'display', 'lat': 25.69063195347653...",25.690632,-80.313706,NaN,33156,FL,Soho Fashion Inc at Dadeland Mall,v-1597447855,NaN
9,"[{'id': '4bf58dd8d48988d171941735', 'name': 'E...",False,4fb6d86e6b74d33d6926eace,7535 N Kendall Dr,US,Miami,United States,NaN,370,"[7535 N Kendall Dr, Miami, FL 33156, United St...","[{'label': 'display', 'lat': 25.68862093670655...",25.688621,-80.315552,NaN,33156,FL,Maroon 5 Payphone Simon Properties / Dadeland ...,v-1597447855,NaN


In [346]:
filtered_columns2 = ['name', 'categories'] + [col for col in dataframe2.columns if col.startswith('location.')] + ['id']
dataframe_filtered2 = dataframe2.loc[:, filtered_columns]

# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['categories']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
dataframe_filtered2['categories'] = dataframe_filtered2.apply(get_category_type, axis=1)

# clean column names by keeping only last term
dataframe_filtered2.columns = [column.split('.')[-1] for column in dataframe_filtered2.columns]

dataframe_filtered2

,name,categories,address,cc,city,country,crossStreet,distance,formattedAddress,labeledLatLngs,lat,lng,postalCode,state,id
0,Dadeland Mall,Shopping Mall,7535 N Kendall Dr,US,Miami,United States,Dadeland Blvd,5,"[7535 N Kendall Dr (Dadeland Blvd), Miami, FL ...","[{'label': 'display', 'lat': 25.69054534138613...",25.690545,-80.312495,33156,FL,4b14ca5cf964a5208da623e3
1,SHRIMP HOUSE - Seafood Pasta - Dadeland Mall,Seafood Restaurant,7535 N Kendall Dr,US,Kendall,United States,NaN,27,"[7535 N Kendall Dr, Kendall, FL 33156, United ...","[{'label': 'display', 'lat': 25.69064, 'lng': ...",25.690640,-80.312280,33156,FL,52694f11498e980ab15d1af8
2,Donald J Pliner Miami - Dadeland Mall,Shoe Store,7535 N Kendall Dr.,US,Miami,United States,NaN,102,"[7535 N Kendall Dr., Miami, FL 33156, United S...","[{'label': 'display', 'lat': 25.6895901, 'lng'...",25.689590,-80.312345,33156,FL,568435fe498e546e328d3958
3,Greenery Mall,Shopping Mall,7700 N Kendall Dr,US,Miami,United States,NaN,793,"[7700 N Kendall Dr, Miami, FL 33156, United St...","[{'label': 'display', 'lat': 25.68800747614737...",25.688007,-80.319910,33156,FL,4bc47d42920eb71304c51e2c
4,Sugar King Dadeland Mall,Candy Store,7535 N Kendall Dr # K148,US,Miami,United States,NaN,6,"[7535 N Kendall Dr # K148, Miami, FL 33156, Un...","[{'label': 'display', 'lat': 25.690559, 'lng':...",25.690559,-80.312484,33156,FL,5e616b8b35ec690008a119a6
5,fit2run dadeland mall,Sporting Goods Shop,NaN,US,Kendall,United States,NaN,48,"[Kendall, FL, United States]","[{'label': 'display', 'lat': 25.69010645150274...",25.690106,-80.312696,NaN,FL,5332f34e498e79e651781035
6,New York & Company - Dadeland Mall,Women's Store,72 Avenue,US,Miami,United States,Kendall drive,94,"[72 Avenue (Kendall drive), Miami, FL 33156, U...","[{'label': 'display', 'lat': 25.691221, 'lng':...",25.691221,-80.312008,33156,FL,59e2afc20d173f42885d36f3
7,Cellairis - Dadeland Mall,Electronics Store,7535 N Kendall Dr.,US,Miami,United States,NaN,182,"[7535 N Kendall Dr., Miami, FL 33156, United S...","[{'label': 'display', 'lat': 25.69046031590772...",25.690460,-80.314322,33156,FL,4e15019c52b1b9e5643ce942
8,Soho Fashion Inc at Dadeland Mall,Shopping Mall,7535 N Kendall Dr,US,Miami,United States,NaN,121,"[7535 N Kendall Dr, Miami, FL 33156, United St...","[{'label': 'display', 'lat': 25.69063195347653...",25.690632,-80.313706,33156,FL,4f1a2db07beb3367ac3b4580
9,Maroon 5 Payphone Simon Properties / Dadeland ...,Event Space,7535 N Kendall Dr,US,Miami,United States,NaN,370,"[7535 N Kendall Dr, Miami, FL 33156, United St...","[{'label': 'display', 'lat': 25.68862093670655...",25.688621,-80.315552,33156,FL,4fb6d86e6b74d33d6926eace


__*NOTE:*__: I specifically searched for the area closest to Dadeland Mall to avoid overcrowding of data.

In [363]:
lat= dataframe2['location.lat']
lng= dataframe2['location.lng']
dataframe_filtered2['lat']= lat
dataframe_filtered2['lng']= lng

/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()
/opt/conda/envs/Python36/lib/python3.6/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [367]:
venues2_map = folium.Map(location=[latitude2, longitude2], zoom_start=15)

for lat, lng, label in zip(dataframe_filtered2.lat, dataframe_filtered2.lng, dataframe_filtered2.name):
    folium.features.CircleMarker(
        [lat, lng],
        radius=5,
        color='blue',
        popup=label,
        fill = True,
        fill_color='blue',
        fill_opacity=0.6
    ).add_to(venues2_map)

# display map
venues2_map